# Computing velocities through lines parallel to the cut

## Load data and modules

In [ ]:
# !{sys.executable} -m pip install import-ipynb

In [35]:
# Load modules
import tqdm
import czifile
import tifffile
import sys
import os
import io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib

from PIL import Image
from IPython.display import Image as IPImage, display
from matplotlib.animation import FuncAnimation
from skimage.io import imread

import cv2


In [ ]:
# Local function import
sys.path.append(os.path.abspath('../..'))
import import_ipynb
from piv.data_exploration import visualize_frames_grid
from piv.points_df import Points

In [2]:
# Convert images to tiff
src_folder = '../../data/ablation-czi'
new_folder = '../../data/tiff'

# Create new folder
os.makedirs(new_folder, exist_ok=True)

# Check if destination folder already has files
if len(os.listdir(new_folder)) > 0:
    print(f"Folder {new_folder} already has files — skipping conversion.")
else:
    # Convert all .czi files
    for file in tqdm.tqdm(os.listdir(src_folder), desc="Converting CZI to TIFF"):
        if file.endswith('.czi'):
            czi_path = os.path.join(src_folder, file)
            out_path = os.path.join(new_folder, file.replace('.czi', '.tiff'))

            with czifile.CziFile(czi_path) as czi:
                img = czi.asarray()
                tifffile.imwrite(out_path, img)

    print("Conversion complete!")

Folder ../../data/tiff already has files — skipping conversion.


In [ ]:
df = Points.from_csv('out/points.csv')
df.head()

In [ ]:
experiment = '220530-E1_Out'

In [ ]:
# Reading in the image from the associated experiment

image_dir = '../../data/tiff'
image_files = [f for f in os.listdir(image_dir) if experiment in f and f.endswith('.tiff')] # type: ignore
if len(image_files) > 1:
    raise ValueError(f'Expected only one image for experiment {experiment}, but found {len(image_files)}')

image = tifffile.TiffFile(f'{image_dir}/{image_files[0]}')
image_data = np.squeeze(image.asarray())

In [ ]:
# Preparation for visual plotting
channel_red     = image_data[0] # shape -> (115, 512, 512)
channel_green   = image_data[1] # shape -> (115, 512, 512)

# Normalize for visualization 
channel_red_norm    = channel_red / channel_red.max()
channel_green_norm  = channel_green / channel_green.max()

# Create RG(B) stack for each slice
rgb_stack = np.zeros((115, 512, 512, 1), dtype=np.float32)
rgb_stack[..., 0] = channel_red_norm
#rgb_stack[..., 1] = channel_green_norm

## Annotate frame with points

In [31]:
# Points csv
cell_id_to_keep = "220914-E4_Out"

# Load the data
points_file_loc = f"../../data/ablation-lineage/{cell_id_to_keep}.lineage"
df = pd.read_csv(points_file_loc, sep='\t')

df.head()


,cell,time,x,y,z,radius,self,parent,child1,child2
0,New nucleus13,3,316.0,211.0,1.0,6.791667,0,-1,0,-1
1,New nucleus7,3,317.0,209.0,0.0,6.791667,1,-1,1,-1
2,New nucleus9,3,225.0,268.0,1.0,6.791667,2,-1,2,-1
3,New nucleus5,3,274.0,259.0,0.0,7.625000,3,-1,3,-1
4,New nucleus11,3,272.0,312.0,1.0,6.791667,4,-1,4,-1
